In [1]:
import sympy,time
from sympy.matrices import Matrix #最后处理矩阵用
#导入工具包
load('pkg/Lie.sage') #李代数相关的工具
load('pkg/spo.sage') #李超代数spo的工具
load('pkg/MatSp.sage') #处理矩阵的工具
load('pkg/AlgebraBSC.sage') #结构常数相关工具
load('pkg/pbw.sage') #李代数pbw基的工具
load('pkg/pbws.sage') #李超代数pbw基的工具
load('pkg/rex.py') #解方程等相关工具
print('''导入相关工具包：
1. Lie.sage：李代数工具，以及C,D族李代数
2. spo.sage：李超代数工具，以及spo
3. MatSp.sage：矩阵空间，求矩阵关于矩阵基的线性表示
4. AlgebraBSC：求李/李超代数的结构常数，以及由结构常数生成代数
5. pbw.sage/pbws.sage 李/李超代数的pbw基
6. rex.py 解方程工具''')

导入相关工具包：
1. Lie.sage：李代数工具，以及C,D族李代数
2. spo.sage：李超代数工具，以及spo
3. MatSp.sage：矩阵空间，求矩阵关于矩阵基的线性表示
4. AlgebraBSC：求李/李超代数的结构常数，以及由结构常数生成代数
5. pbw.sage/pbws.sage 李/李超代数的pbw基
6. rex.py 解方程工具


In [2]:
#V初始化数据
tt = time.time()
m,n = 1,3
L = SpO(m,n) #李超代数spo(2m|2n)
mats = L.negative_matrixs + L.h_matrixs + L.positive_matrixs #矩阵信息
pos_m = len(L.positive_roots)
print('李超代数：spo(%d,%d)\n空间维数为：%d\n正根数目为：%d'%(2*m,2*n,len(mats),pos_m))
es = Lie.symbols('e',pos_m)
fs = Lie.symbols('f',pos_m)
hs = Lie.symbols('h',m+n) 
syms = fs + hs +es #用作结构常数的符号变量
#奇根对应的符号变量
roots = L.positive_roots 
ind1 = [roots.index(r) for r in L.odd_roots if r in roots]
roots = L.simple_roots
ind2 = [roots.index(r) for r in L.odd_roots if r in roots]
odds = [fs[i] for i in ind1] + [hs[i] for i in ind2] + [es[i] for i in ind1] 

#计算结构常数
print('计算结构常数')
N = AlgebraBSC.lie_sup_SC(mats,L.odd_mats)
'''
Lf = {i+1:Element(element,N,syms) for i,element in enumerate(fs)}
Le = {i+1:Element(element,N,syms) for i,element in enumerate(es)}
Lh = {i+1:Element(element,N,syms) for i,element in enumerate(hs)}
#'''
#pbw基
print('计算李超代数泛包络的pbw基')
vf = {i+1:SPBWElement(element,N,syms,odds) for i,element in enumerate(fs)}
ve = {i+1:SPBWElement(element,N,syms,odds) for i,element in enumerate(es)}
vh = {i+1:SPBWElement(element,N,syms,odds) for i,element in enumerate(hs)}
ide = SPBWElement({tuple():1},N,syms,odds) #幺元
vectors = L.vectors
print('初始化完毕，用时 %.3fs'%(time.time()-tt))

李超代数：spo(2,6)
空间维数为：30
正根数目为：13
计算结构常数
计算李超代数泛包络的pbw基
初始化完毕，用时 7.821s


In [3]:
#权的正根分解
tt = time.time()
s,t = 1,1
weight = L.basis[s-1] + L.basis[m+t-1];print('待讨论的权：',weight) #待讨论的权（正交基形式）
root = L.vector2root(weight) #转单根形式
dec = L.positive_dec(root) #正根分解（元组构成）
negative_roots = L.negative_roots #负根
dec_root = []
for line in dec:
    new = []
    for i,element in enumerate(line):
        if element:
            new = new + [negative_roots[i]]*element
    dec_root.append(tuple(new))
dec_vector = [tuple(L.root2vector(r) for r in line) for line in dec_root]
print('权空间维数：',len(dec)) #分解总数目
print('用时：%.3fs'%(time.time()-tt))
#for line in res_vectors:print(line)
table([[i for i in line if i] for line in dec_vector]) #分解结果

待讨论的权： delta + epsilon1
权空间维数： 15
用时：0.692s


-delta + epsilon1
  -delta + epsilon1
  -epsilon1 + epsilon2
  -delta + epsilon1
  -epsilon1 + epsilon2
  -delta + epsilon1
  -delta + epsilon1
  -epsilon2 + epsilon3
  -epsilon2 - epsilon3
  -epsilon1 + epsilon2
  -epsilon1 + epsilon3
  -delta + epsilon2
  -epsilon1 + epsilon2
  -epsilon1 - epsilon3
  -delta - epsilon1

In [4]:
#相应的pbw基
basis = [tuple(fs[negative_roots.index(i)] for i in line) for line in dec_root]
pbw_basis = [ide.tuple2element(i) for i in basis]
print('权空间对应的pbw基：')
pbw_basis

权空间对应的pbw基：


[{(f1, f2, f3): 1}, {(f2, f5): 1}, {(f3, f4): 1}, {(f6,): 1}]

In [5]:
#最高权向量
x = Lie.symbols('x',n)
z = list(Lie.symbols('z',m))
z[s-1] = x[t-1]+2*n-s-t
la = sum([i*j for i,j in zip(z,L.basis[:m])]) + sum([i*j for i,j in zip(x,L.basis[m:])])
#d_s+e_t: a_s = b_t+2n-s-t
print('最高权：',la)

#泛包络上计算结果res
tt = time.time()
res = [[ve[i]*element for element in pbw_basis] for i in range(1,m+n+1)]
print("计算泛包络上的乘积结果，乘积总数：",(m+n)*len(pbw_basis))
print('用时：%.3fs\n'%(time.time()-tt))

#化简为Verma模的元素
tt = time.time()
res_verma = [[i.to_verma(la,L,m+n) for i in line] for line in res]
print('转化为Verma模上结果，用时：%.3fs\n'%(time.time()-tt))

#转化为矩阵
tt = time.time()
data = []
for line in res_verma:
    mat,_ = ide.linear_rep(line)
    data.append(mat)
mat = matrix.block(m+n,1,data)
mat_sym = Matrix(mat)
if m<=3:
    print('矩阵的秩为',mat_sym.rank()) #54打印时间太长了！
print('矩阵阶数为 %d*%d  '%mat.dimensions(),end='')
print('计算用时：%.3fs'%(time.time()-tt))
#save_vari(mat_sym,'mat%d.pydata'%k) #保存数据

最高权： delta*(x1 + 2) + epsilon1*x1 + epsilon2*x2
计算泛包络上的乘积结果，乘积总数： 12
用时：0.732s

转化为Verma模上结果，用时：0.412s

矩阵的秩为 3
矩阵阶数为 5*4  计算用时：0.010s


In [7]:
latex(mat)

\left(\begin{array}{rrrr}
2 \, x_{1} & 1 & 1 & 0 \\
\hline
 0 & x_{1} - x_{2} + 1 & 1 & -1 \\
x_{1} - x_{2} & 0 & 1 & 0 \\
\hline
 x_{1} + x_{2} & 1 & 0 & 0 \\
0 & 1 & x_{1} + x_{2} + 1 & -1
\end{array}\right)

In [35]:

    for i in range(4):
        a = Matrix([mat_sym.col(j).transpose() for j in range(4) if j!=i])
        print(a.rank())

3
3
3
3


In [30]:
a = Matrix([mat_sym.col(i) for i in range(3)])

In [9]:
#过渡矩阵
Lie.symbols('x',2)
f = {i+1:j for i,j in enumerate(fs)}
res = [ 1,- x1 - x2, x2 - x1, - x1^2 - 2*x1 + x2^2]
res = matrix(res).T
res2 = []
for i1 in [(1,2,3),(1,3,2),(2,1,3),(2,3,1),(3,1,2),(3,2,1)]:
    for i2 in [(2,5),(5,2)]:
        for i3 in [(3,4),(4,3)]:
            roots = [i1,i2,i3,(6,)]
            roots = [tuple(f[i] for i in line) for line in roots] #转字符串
            roots = [ide.tuple2element(i) for i in roots] #转pbw元素
            keys = [list(i.keys)[0] for i in pbw_basis] #键值
            mat = matrix([[root.element[key] if key in root else 0 for root in roots] for key in keys])
            new_res = mat.inverse()*res
            res2.append([factor(j) for i,j in enumerate(new_res.column(0))])

In [11]:
latex(table(res2))

\begin{tabular}{llll}
$1$ & $-x_{1} - x_{2}$ & $-x_{1} + x_{2}$ & $-x_{1}^{2} + x_{2}^{2} - 2 \, x_{1}$ \\
$1$ & $-x_{1} - x_{2}$ & $-x_{1} + x_{2}$ & $-{\left(x_{1} + x_{2}\right)} {\left(x_{1} - x_{2} + 1\right)}$ \\
$1$ & $-x_{1} - x_{2}$ & $-x_{1} + x_{2}$ & $-{\left(x_{1} + x_{2} + 1\right)} {\left(x_{1} - x_{2}\right)}$ \\
$1$ & $-x_{1} - x_{2}$ & $-x_{1} + x_{2}$ & $-{\left(x_{1} + x_{2}\right)} {\left(x_{1} - x_{2}\right)}$ \\
$1$ & $-x_{1} - x_{2}$ & $-x_{1} + x_{2}$ & $-x_{1}^{2} + x_{2}^{2} - 2 \, x_{1}$ \\
$1$ & $-x_{1} - x_{2}$ & $-x_{1} + x_{2}$ & $-{\left(x_{1} + x_{2}\right)} {\left(x_{1} - x_{2} + 1\right)}$ \\
$1$ & $-x_{1} - x_{2}$ & $-x_{1} + x_{2}$ & $-{\left(x_{1} + x_{2} + 1\right)} {\left(x_{1} - x_{2}\right)}$ \\
$1$ & $-x_{1} - x_{2}$ & $-x_{1} + x_{2}$ & $-{\left(x_{1} + x_{2}\right)} {\left(x_{1} - x_{2}\right)}$ \\
$1$ & $-x_{1} - x_{2}$ & $-x_{1} + x_{2} - 1$ & $-{\left(x_{1} + x_{2} + 1\right)} {\left(x_{1} - x_{2} + 1\right)}$ \\
$1$ & $-x_{1} - x_{2}$ &

""


In [51]:
table(res2)

,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
